# The Unreasonable Effectiveness of Character-level Language Models
# (and why RNNs are still cool)

## By [Yoav Goldberg](http://www.cs.biu.ac.il/~yogo) (2015)

#### (with minor changes by Peter Norvig (2022) for modern Python 3)

<hr>

[RNNs](https://en.wikipedia.org/wiki/Recurrent_neural_network), [LSTMs](https://en.wikipedia.org/wiki/Long_short-term_memory) and [Deep Learning](https://en.wikipedia.org/wiki/Deep_learning) are all the rage, and a recent [blog post](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) by Andrej Karpathy is doing a great job explaining what these models are and how to train them.
It also provides some very impressive results of what they are capable of.  This is a great post, and if you are interested in natural language, machine learning or neural networks you should definitely read it. 

Go [**read it now**](http://karpathy.github.io/2015/05/21/rnn-effectiveness/), then come back here. 

You're back? Good. Impressive stuff, huh? How could the network learn to imitate the input like that?
Indeed. I was quite impressed as well.

However, it feels to me that most readers of the post are impressed by the wrong reasons.
This is because they are not familiar with **unsmoothed maximum-liklihood character level language models** and their unreasonable effectiveness at generating rather convincing natural language outputs.

In what follows I will briefly describe these character-level maximum-likelihood langauge models, which are much less magical than RNNs and LSTMs, and show that they too can produce a rather convincing Shakespearean prose. I will also show about 30 lines of python code that take care of both training the model and generating the output. Compared to this baseline, the RNNs may seem somehwat less impressive. So why was I impressed? I will explain this too, below.

## Unsmoothed Maximum Likelihood Character Level Language Model 

The name is quite long, but the idea is very simple.  We want a model whose job is to guess the next character based on the previous *n* characters. For example, having seen `ello`, the next characer is likely to be either a commma or space (if we assume is is the end of the word "hello"), or the letter `w` if we believe we are in the middle of the word "mellow" or "yellow". Humans are quite good at this, but of course seeing a larger history makes things easier (if we were to see 5 letters instead of 4, the choice between space and `w` would have been much easier).

We will call *n*, the number of letters we need to guess based on, the _order_ of the language model.

RNNs and LSTMs can potentially learn infinite-order language model (they guess the next character based on a "state" which supposedly encodes all the previous history). We here will restrict ourselves to a fixed-order language model.

So, we are seeing *n* letters, and need to guess the *n+1*th one. We are also given a large-ish amount of text (say, all of Shakespeare's works) that we can use. How would we go about solving this task?

Mathematically, we would like to learn a function *P(c* | *h)*. Here, *c* is a character, *h* is a *n*-character history, and *P(c* | *h)* stands for how likely is it to see *c* after we've seen *h*.

Perhaps the simplest approach would be to just count and divide (a.k.a **maximum likelihood estimates**). We will count the number of times each letter *c* appeared after *h*, and divide by the total numbers of letters appearing after *h*. The **unsmoothed** part means that if we did not see a given letter following *h*, we will just give it a probability of zero.

And that's all there is to it.

## Training Code

Here is the code for training a language model, which implements *P(c* | *h)* with a counter of the number of times we have seen each character, for each history. The function `train_char_lm` takes a filename  to read the characters from. `order` is the history size to consult. Note that we pad the data with `order` leading characters so that we also learn how to start.


In [ ]:
import random
import collections

In [ ]:
class LanguageModel(collections.defaultdict):
    """A mapping from `order` history characters to possible next characters and their 
    frequency, e.g. {'spea': Counter({'k': 9, 'r': 1})} lets us generate 'speak' or 'spear'."""
    def __init__(self, order): 
        self.order = order
        self.default_factory = collections.Counter 

def train_char_lm(fname, order=4) -> LanguageModel:
    """Train an character-level language model of given order on all the text in `fname`."""
    lm = LanguageModel(order)
    data = (order * PAD) + open(fname).read()
    for i in range(order, len(data)):
        history, char = data[i - order:i], data[i]
        lm[history][char] += 1
    for counter in lm.values():
        counter.total = sum(counter.values()) # Cache total counts (for sample_character)
    return lm

PAD = '`' # Character to pad the beginning of a text

Let's train a model on Andrej's Shakespeare text:

In [3]:
! [ -f shakespeare_input.txt ] || curl -O https://norvig.com/ngrams/shakespeare_input.txt
! wc shakespeare_input.txt

  167204  832301 4573338 shakespeare_input.txt


In [4]:
lm = train_char_lm("shakespeare_input.txt", order=4)

Ok. Now let's do some queries on the language model:

In [5]:
lm['ello']

Counter({'r': 35,
         'w': 480,
         'u': 22,
         ',': 16,
         ' ': 8,
         '.': 4,
         '?': 4,
         ':': 3,
         'n': 1,
         "'": 10,
         '!': 4})

In [6]:
lm['Firs']

Counter({'t': 864})

In [7]:
lm['rst ']

Counter({'C': 119,
         'f': 14,
         'i': 21,
         't': 67,
         'u': 2,
         'S': 203,
         'h': 24,
         's': 41,
         'R': 1,
         'b': 31,
         'c': 16,
         'O': 23,
         'w': 30,
         'a': 28,
         'm': 28,
         'n': 25,
         'I': 12,
         'L': 133,
         'M': 74,
         'l': 13,
         'o': 38,
         'H': 5,
         'd': 19,
         'W': 42,
         'K': 10,
         'q': 2,
         'G': 112,
         'g': 14,
         'k': 5,
         'e': 4,
         'y': 3,
         'r': 9,
         'p': 11,
         'A': 7,
         'P': 18,
         'F': 15,
         'v': 3,
         'T': 4,
         'D': 4,
         'B': 12,
         'N': 1,
         "'": 1,
         'E': 2})

So `"ello"` is followed by either space, punctuation or `w` (or `r`, `u`, `n`), `"Firs"` is pretty much deterministic, and the word following `"rst "` can start with pretty much every letter. 

## Character Probabilities

We can extract probabilities *P(c* | *h)* from the model as follows:

In [8]:
def P(c, h, lm) -> float: 
    """The probability P(c | h) of next character c given history h, according to the language model."""
    return lm[h][c] / lm[h].total

In [9]:
P('w', 'ello', lm)

0.817717206132879

In [10]:
P('t', 'Firs', lm)

1.0

In [11]:
P('x', 'Firs', lm)

0.0

In [12]:
P('S', 'rst ', lm)

0.16292134831460675

## Generating sample text from a model

To randomly generate a sample text from a model, we maintain a history of *order* characters, starting  with all pad characters. We then enter a loop that looks up the history in the language model, randomly samples a character from the history's counter, then updates the history by dropping its first character and adding the randomly-sampled character to the end. 

In [13]:
def generate_text(lm, length=1000) -> str:
    """Sample a random `length`-long passage from `lm`."""
    history = lm.order * PAD
    text = []
    for _ in range(length):
        c = sample_character(lm[history])
        history = history[1:] + c
        text.append(c)
    return ''.join(text)

To sample a single character *c* from a counter, randomly choose an integer *n* from 1 to the total count of characters in the counter, then iterate through the counter until the cumulative total of the counts meets or exceeds *n*.

In [14]:
def sample_character(counter) -> str:
    """Randomly sample the nth character from the counter."""
    n = random.randint(1, counter.total)
    cumulative = 0
    for c in counter:
        cumulative += counter[c]
        if cumulative >= n: 
            return c

## Generating Shakespeare text from different order models

Let's try to generate text based on different language-model orders. Let's start with something silly:

## Order 2:

In [15]:
lm = train_char_lm("shakespeare_input.txt", order=2)
print(generate_text(lm))

Fir,
ACBENE:
But st thaverforn'd his then my ther's one,
Whe Lonce de kinger, ap the sa? Whee mus ne.

But of ithice soaciven loven one my I ch hight sithe or ble-ern the my wherephy graidia,
a belf;
Ay, quare hins an an:
Whem'd cur beek, Hecomat on my he arry Riciuse, giver nothery: nal.

DUKE:
Hold but,
And to moul ford
ant ace
wore hime re shathis onsol th I dis spried!

SIR Lorthat mor ent: sur suchast the spoo, wit, the in hief-wis hand lor reand way be wo set this solorsee Dand, loot the makin am hisfourab:
Bar his the frortithe uponowee:
Is han hour leyessin on to and take my pas and to alset hatterialow
Suchaso you
be creest to tak alks.

PATIANTIS Fraff,
ime proier theatholk yould a the raingrain anith, th hardso I halmorge th wall wast liver,
in theall-dayst this craire swot Pere whow And nobeece
Tithall, ar,
Thar und of ch mandrefor the
Andes, Dest allighe an:
Warrow thavererstund no lad your som of why nothat my whows be pur hat chou!

SLYCUS:
It th withith it pon spe myse 

## Order 4 

Order 2 was not so great... but what if we increase the order to 4?



In [16]:
lm = train_char_lm("shakespeare_input.txt", order=4)
print(generate_text(lm))

First Murderer:
Unquet thee one; and chant:
Birons--
Poor cheeks they much my bosom sorrow! how stops as him your honest,
And it better? Rugby, if you pleasures, all me writter of men.

EGLAMOUR:
Ay, but him a tready the work madam?

MENENIUS:
Lamentony conquer'd in purpose; above.

SIR ANDRONICUS:
And be my pardon my father for ink, he's greaten him bound thus?
Let thee,
And lady, willius' tongue deed world.

PRINCE EDWARD:
As love hold, to be constantiately captains o' thee, purs and Petructified: but his judgment.

THUR:
O Lord of thou the bold not as well take thus sends--takings thready friends, but that he sleep o' the neverended.

MACDUFF:
Venice that blasp and him was faults struth the more of men so well;
Ford, I should betray in this in the bear you,
To sand I hadst travery,
Yet, herrily;
And bed to king: God sense
Caius with and still behind.

SUFFOLK:
Right a tear
shout of suborned so prince him. Your titless'd the better is be the she, swell!
This a little come, and sons;




## Order 7

Order 4 is already quite reasonable, and reads like English. Just 4 letters history! What if we increase it to 7?

In [17]:
lm = train_char_lm("shakespeare_input.txt", order=7)
print(generate_text(lm))

First Citizen:
Before I keep the
sounder a rhyme nor pinch the engenders you:
And he was the valour that blows, they did;
So I grow; I prospero my lords,
Will creep into strong,
And I'll rake up, pipers.

FALSTAFF:
Fie! you played the midwife present a large and grave,
Being simply; the honour,
Purchase the good hope
The period to make us medicine of kindness: I dare not making?

GONZALO:
All's hush'd with the vast sea: at land:
Come, come, come, though, haply, are you colours turn'd youth: who best king of though
The enmity.

ORLEANS:
He's alive, I was not angry indeed to him.

MENENIUS:
I loved withal: except a sword, despite of brooded waters, and
that all tire.

WARWICK:
Ay, sir, an she to be doubtful for those that
it will have my country's wreck, to transportance;
Sometimes, like a rebel,
And dreadful object him, till the fashion? do I not known
No less home, you wrestler's
heels a huge to be impart to hide thee much to live created one of you; which marriage move
And she is fast

## How about order 10?

In [18]:
lm = train_char_lm("shakespeare_input.txt", order=10)
print(generate_text(lm))

First Citizen:
Read the will.

ANTONY:
He will seek
Some way to leave as keep; whose top to climb
Is certain and shameless callet know herself.
Helen of Greece! what should I curse the ducats.'

SALARINO:
That's not my meaning:
go to thy cost.

VERNON:
There is no force in the first, how get hence:
Why should the gods to witness from their colour fly,
And to become the function takes,
The ear more quick of apprehensions, motions,
as promising
Than a wild dedication; facere, as it grows.

Poet:
Ay, that I can do it: I
commend you well, my lord,
Grievous complaint; hasty and tinder-like
upon too trivial motion; one that, in King Edward's good success hath done to-day
Mad and fantastical banquet, just so much they love his lady was but devised at first, to try her skill,
Reignier, whose frank heart gave all,--
O, that way and you to your majesty had call'd you up, have held him dear.

BEVIS:
Come, and believe thee,
Were they not by you?

LENNOX:
Ay, my good lord;
'Tis but the gods to inte

## This works pretty well

With an order of 4, we already get quite reasonable results. Increasing the order to 7 (about a word and a half of history) or 10 (about two short words of history) already gets us quite passable Shakepearan text. I'd say it is on par with the examples in Andrej's post. And how simple and un-mystical the model is!

## Aside: First words

One thing you may have noticed: all the generated passages start with "Fi". Why is that? Because the training data starts with the word "First" (preceeded by padding), and so when we go to randomly `generate_text`, the only thing that follows the initial padding is the word "First". We could get more variety in the generated text by breaking the training text up into sections  and inserting padding at the start of each section. But that would require some knowledge of the structure of the training text; right know the only assumption is that it is a sequence of characters.

In [19]:
! head shakespeare_input.txt

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:


## So why am I impressed with the RNNs after all?

Generating English a character at a time -- not so impressive in my view. The RNN needs to learn the previous *n* letters, for a rather small *n*, and that's it. 

However, Karpathy's C++ code generation example is very impressive. Why? because of the context awareness. Note that in all of Karpathy's posted examples, the code is well indented, the braces and brackets are correctly nested, and even the comments start and end correctly. This is not something that can be achieved by simply looking at the previous *n* characters. 

If Karpathy's examples are not cherry-picked, and the output is generally that nice, then the LSTM did learn something not trivial at all.

# Linux Kernel C++ Code

Just for the fun of it, let's see what our simple language model does with the Linux-kernel code:

In [20]:
! [ -f linux_input.txt ] || curl -O https://norvig.com/ngrams/linux_input.txt
! wc linux_input.txt

  241465  759639 6206997 linux_input.txt


In [21]:
lm = train_char_lm("linux_input.txt", order=10)
print(generate_text(lm))

/*
 * linux/kernel.h>
#include <linux/module.h>
#include <linux/proc_fs.h>
#include <linux/sched/rt.h>
#include "rcu.h"

MODULE_AUTHOR("Paul E. McKenney <paulmck@us.ibm.com), IBM Corp. 2006
 *       - If the kernel
 * interface for test commands. */
static int set_tracing_thresh > GRAPH_MAX_FUNC_TEST) {
		trace_array_put(this_tr);
	mutex_unlock_pi;

	/*
	 * One idle CPUs. */
	shuffle_intervals
	 * (think "ticks") worth of data
 */
static int func_id)
{
	switch (ret) {
			if (group_rq && (rt_rq_throttled;
}

/**
 * worker_clr_flags(worker, WORKER_PREP);
sleep:
	/*
	 * In the semi idle case by checking on the CPU, it can't propagate the requeue
 * @mode:	expiry mode: absolute time */
	if (ftrace_enabled) {
		/* Keep track of cpu being initialization of architecture-specific fields. */
	add_taint(TAINT_LIVEPATCH\n");
	add_taint(TAINT_FORCED_MODULE))
		buf[l++] = 'E';

	rwbs[i] = '\0';
	if (count > 0) {
		error = -EINVAL;
	}

	if (prctl_map->__m1 __op				\
	 (unsigned long ticks)
{
	jiffie

In [22]:
lm = train_char_lm("linux_input.txt", order=15)
print(generate_text(lm))

/*
 * linux/kernel/timer.c.
 *  Please see that file for copyright and history logs.
 *
 */

#include <linux/slab.h>
#include <linux/kernel.h>
#include <linux/sysrq.h>
#include <linux/fs.h>
#include <linux/sched/sysctl.h>
#include <linux/percpu-rwsem.h>
#include <linux/utsname.h>
#include <linux/hardirq.h>
#include <linux/tty.h>
#include <linux/hardirq.h>
#include <linux/irq.h>
#include <linux/atomic.h>
#include <linux/skbuff.h>
#ifdef CONFIG_DEBUG_OBJECTS_RCU_HEAD kernel builds.
 */
void destroy_rcu_head_on_stack(&rh1);
	init_rcu_head_on_stack(&rcu.head);
	init_completion(&self.exited);
	init_completion(&rcu.completion);

	/* Other rcu_barrier(). */
	/* End of fields guarded by barrier_mutex. */

	atomic_long_t refs;
	struct rcu_node *rnp;

	if (t->rcu_read_lock_nesting, shared state will be updated only when filter_hash updating */
		ret = atomic_add_return(1, &rttest_event);
		td->mutexes[td->opdata] = 1;
		td->event = atomic_add_return(0, &rdp->dynticks->dynticks_nesting, rdtp->dyn

In [23]:
lm = train_char_lm("linux_input.txt", order=20)
print(generate_text(lm))

/*
 * linux/kernel/irq/manage.c
 *
 * Copyright (C) 2006 Timesys Corp., Thomas Gleixner
 *
 * This code is licenced under the GPL version 2. For details see
 * kernel-base/COPYING
 */

#include <linux/mm.h>
#include <linux/binfmts.h>
#include <linux/spinlock.h>
#include <linux/sched.h> /* for spin_unlock_irq() using preempt_count() m68k */
#include <linux/types.h>
#include <linux/mutex.h>
#include <linux/fs.h>
#include <linux/module.h>
#include <linux/nfs_fs.h>
#include <linux/percpu.h>
#include <linux/slab.h>

#include "power.h"

static DEFINE_MUTEX(stop_cpus_mutex);
static DEFINE_SPINLOCK(rcu_torture_lock);
static DEFINE_PER_CPU(struct llist_head, call_single_queue);

static void flush_module_icache(const struct module *mod,
			 const unsigned long seccomp_mode = SECCOMP_MODE_STRICT:
		op = SECCOMP_SET_MODE_FILTER:
		return __seccomp_phase1_filter(int this_syscall, struct seccomp_data *sd)
{
	struct seccomp_filter *walker;

	assert_spin_locked(&current->sighand->siglock);
	set_proces

In [24]:
print(generate_text(lm))

/*
 * linux/kernel/irq/autoprobe.c
 *
 * Copyright (C) 2003-2004 Amit S. Kale <amitkale@linsyssoft.com>
 * Copyright (C) 2012 Red Hat, Inc. All Rights Reserved.
 *
 * This program is free software; you can redistribute it and/or modify
 * it under the terms of the GNU General Public License version 2 as
 * published by the Free Software Foundation.
 */

#include <linux/sched/rt.h>

#include "trace.h"
#include "trace_output.h"

struct header_iter {
	struct pci_dev *dev;
};

static struct rb_root swsusp_extents = RB_ROOT;

static int swsusp_page_is_forbidden(page) && swsusp_page_is_free(virt_to_page(lp))) {
			/* The page is "safe", add it to the list */
			lp->next = safe_pages_list;
		safe_pages_list = safe_pages_list->next;
	pbe->next = restore_pblist;
	restore_pblist = NULL;
	buffer = NULL;
	alloc_normal = 0;
	alloc_highmem = 0;

	/* Count the number of saveable data pages. */
	save_highmem = count_highmem_image_pages - compute the total number of overruns from
 */
unsigned long
ring

In [25]:
print(generate_text(lm, length=5000))

/*
 * linux/kernel/irq/resend.c
 *
 * Copyright (C) 1997  Andrew Main <zefram@fysh.org>
 *
 * Integrated into 2.1.97+,  Andrew G. Morgan <morgan@kernel.org>
 * 30 May 2002:	Cleanup, Robert M. Love <rml@tech9.net>
 */

#define pr_fmt(fmt) KBUILD_MODNAME ": " fmt

#include <linux/module.h>
#include <linux/slab.h>
#include "cpudeadline.h"
#include "cpuacct.h"

struct rq;
struct cpuidle_state *idle_state)
{
	rq->idle_state = idle_state;
}

static inline struct cpuacct *css_ca(struct cgroup_subsys *ss;
	int i;

	init_cgroup_root(&cgrp_dfl_root, &opts);
	cgrp_dfl_root.cgrp.self.flags |= CSS_NO_REF;

	if (early) {
		/* allocation can't be done safely during early init */
		css->id = 1;
	} else {
		css->id = cgroup_idr_alloc(&ss->css_idr, css, 1, 2,
						   GFP_KERNEL);
	if (!data)
		goto out;

	cd->fp = fp;
	cd->clk = get_posix_clock(fp);
	int err = -ENODEV;

	if (cnt >= PAGE_SIZE)
		return -EINVAL;
	rcu_read_lock();
	sd = rcu_dereference(per_cpu(sd_asym, cpu), sd);
}

/*
 * Attach the domai

## Analysis

Order 10 is pretty much junk. In order 15 things sort-of make sense, but we jump abruptly between the `[sic]`
and by order 20 we are doing quite nicely — but are far from keeping good indentation and brackets. 

How could we? we do not have the memory, and these things are not modeled at all. While we could quite easily enrich our model to support also keeping track of brackets and indentation (by adding information such as "have I seen ( but not )" to the conditioning history), this requires extra work, non-trivial human reasoning, and will make the model significantly more complex. 

Karpathy's LSTM, on the other hand, seemed to have just learn it on its own. And that's impressive.